# Test the whole processing of an arbitrary input

In [2]:
import csv
import pickle

import pandas as pd
import lda

## import the statistics info of the database

In [3]:
movie = pd.read_csv('5000data.csv')
directors = list(movie.loc[:,'director_name'])
temp = list(movie.loc[:,'director_name'])
directors_likes = [840 if temp[i]==0 else temp[i] for i in range(len(temp))]
l1 = list(movie.loc[:,'actor_1_name'])
l10 = list(movie.loc[:,'actor_1_facebook_likes'])
l2 = list(movie.loc[:,'actor_2_name'])
l20 = list(movie.loc[:,'actor_2_facebook_likes'])
l3 = list(movie.loc[:,'actor_3_name'])
l30 = list(movie.loc[:,'actor_3_facebook_likes'])
actors = dict(zip(l1 + l2 + l3, l10 + l20 + l30))

stat = pd.read_csv('statistics.csv')

## make up a movie entry

### basic info

In [44]:
duration = stat.loc[0, 'duration']
budget = 3*10^8

### Cast

In [45]:
director_facebook_likes = stat.loc[0, 'director_facebook_likes']
actor_3_facebook_likes = stat.loc[0, 'actor_3_facebook_likes']
actor_2_facebook_likes = stat.loc[0, 'actor_2_facebook_likes']
actor_1_facebook_likes = stat.loc[0, 'actor_1_facebook_likes']
cast_total_facebook_likes = stat.loc[0, 'cast_total_facebook_likes']

### Review

In [46]:
movie_facebook_likes = stat.loc[0, 'movie_facebook_likes']
num_critic_for_reviews = stat.loc[0, 'num_critic_for_reviews']
num_user_for_reviews = stat.loc[0, 'num_user_for_reviews']
num_voted_users = stat.loc[0, 'num_voted_users']
imdb_score = stat.loc[0, 'imdb_score']

### Content

In [8]:
description = ['alien', 'zombie', 'war', 'planet', 'fear', 'disaster', 'death', 'humanity']

# Action,Adventure,Fantasy,Sci-Fi,Thriller,
# Romance,Animation,Comedy,Family,Musical,
# Mystery,Western,Drama,History, Sport
# Crime,Horror,War,Biography,Music
# Documentary
genre = [0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0]

#### Compute the topic vector

In [9]:
model = pickle.load(open("lda.pkl","rb"))

In [25]:
dict_ = []
with open('dictionary.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        dict_ = row

In [26]:
vec = []
count = 0
for i in range(len(dict_)):
    if dict_[i] in description:
        vec += [1]
        count += 1
    else:
        vec += [0]
count

6

In [28]:
movie = dict(zip(dict_, vec))
movie = pd.DataFrame(movie, index=[0])
movie = movie.values
topic = model.transform(movie)

In [29]:
topic = topic.tolist()[0]
topic

[0.0004693045775345361,
 0.00043845481245066107,
 0.0003914644936237958,
 0.0005202316533699878,
 0.0004934575656496608,
 0.0007603563684332934,
 0.00046015466140852745,
 0.00047002351291695394,
 0.00044161016151573947,
 0.057793553831633475,
 0.0004926652124104733,
 0.0004448112563802296,
 0.18228660965071913,
 0.15811834107577208,
 0.00046787328487550074,
 0.0005098576660740829,
 0.0004108599030023417,
 0.0004879640215764817,
 0.0004982657311852644,
 0.0004903033476540721,
 0.04860694971505372,
 0.0005264804505591966,
 0.0004848794261932499,
 0.0005073284977876552,
 0.34067731874021323,
 0.03807587347712969,
 0.00045268658555013,
 0.00042450250959286627,
 0.000514129456696857,
 0.16328368835303697]

### Combine them into a complete feature vector

In [48]:
feature = [
    num_critic_for_reviews,
    duration,
    director_facebook_likes,
    actor_3_facebook_likes,
    actor_1_facebook_likes,
    num_voted_users,
    cast_total_facebook_likes,
    num_user_for_reviews,
    budget,
    actor_2_facebook_likes,
    imdb_score,
    movie_facebook_likes]
feature += topic

### Preprocessing

In [42]:
## test with a training entry
meaning = pd.read_csv('meaningful.csv')
binned = pd.read_csv('binned.csv')
n = 1024
feature = meaning.loc[n, 'num_critic_for_reviews':'topic29']
del feature['gross']
genre = meaning.loc[n, 'Action':'Documentary']
binned.loc[n, 'gross']

2.0

In [43]:
feature -= stat.loc[0]
feature /= stat.loc[1]

feature -= stat.loc[2]
feature /= stat.loc[3] - stat.loc[2]
feature *= 2
feature -= 1.0

# feature = feature.append(pd.DataFrame(genre)).T
feature = pd.concat([feature, genre])

In [44]:
model = pickle.load(open('decision_tree.pkl', 'rb'))
result = model.predict(feature)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [45]:
result

array([ 2.])

In [57]:
features = list(feature.columns)

In [58]:
import operator
imp = list(model.feature_importances_)
imp = dict(zip(range(63), imp))
sorted_x = sorted(imp.items(), key=operator.itemgetter(1))
sorted_x.reverse()
for i in range(10):
    print(features[sorted_x[i][0]])

budget
topic9
topic5
topic29
topic18
num_voted_users
num_user_for_reviews
topic4
director_facebook_likes
cast_total_facebook_likes
